In [ ]:
!pip install -q flask flask_cors pyngrok transformers accelerate bitsandbytes shap lime scikit-learn numpy

In [ ]:
pip install shap==0.46.0 "numpy<2" "scikit-learn>=1.3,<1.6"

-----------------------------------------

In [ ]:
# ============================================================
# Kaggle Backend for Press Conference Simulator
# ============================================================
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Dict, Any, List
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ============================================================
# 0️⃣ Optional dependencies
# ============================================================
try:
    import shap
except Exception:
    shap = None
try:
    from lime.lime_text import LimeTextExplainer
except Exception:
    LimeTextExplainer = None


# ============================================================
# 1️⃣ Explainability helpers
# ============================================================
def _split_sentences(text: str) -> List[str]:
    parts = [p.strip() for p in text.replace("!", ".").replace("?", ".").split(".")]
    return [p for p in parts if p]


def _semantic_explain(speech: str, question: str) -> str:
    segments = _split_sentences(speech)
    if not segments:
        return "No speech segments to analyze."
    corpus = segments + [question]
    vec = TfidfVectorizer(stop_words="english")
    X = vec.fit_transform(corpus)
    sims = cosine_similarity(X[-1], X[:-1]).ravel()
    if sims.size == 0:
        return "No similarity signal detected."
    order = np.argsort(sims)[::-1][:2]
    tops = [f"• \"{segments[i]}\" (sim={sims[i]:.2f})" for i in order if sims[i] > 0]
    return "Likely influential speech parts:\n" + ("\n".join(tops) if tops else "No strong matches.")


def _avg_logprob_for_target(model, tokenizer, context: str, target: str) -> float:
    """Computes mean log-prob of generating target given context."""
    import torch
    model_device = next(model.parameters()).device
    with torch.no_grad():
        full = context + ("\n" if context and not context.endswith("\n") else "") + target
        enc = tokenizer(full, return_tensors="pt")
        input_ids = enc["input_ids"].to(model_device)
        attn_mask = enc.get("attention_mask", None)
        if attn_mask is not None:
            attn_mask = attn_mask.to(model_device)
        ctx_ids = tokenizer(context, return_tensors="pt")["input_ids"].to(model_device)
        ctx_len = ctx_ids.shape[1]
        outputs = model(input_ids=input_ids, attention_mask=attn_mask)
        logits = outputs.logits
        logprobs = logits.log_softmax(dim=-1)
        target_token_ids = input_ids[:, ctx_len:]
        prev_positions = logprobs[:, ctx_len - 1: -1, :] if ctx_len > 0 else logprobs[:, :-1, :]
        seq = min(prev_positions.shape[1], target_token_ids.shape[1])
        tok_logprobs = prev_positions[:, -seq:, :].gather(dim=-1, index=target_token_ids[:, :seq].unsqueeze(-1)).squeeze(-1)
        return float(tok_logprobs.mean().item())


def _shap_explain(speech: str, question: str, model, tokenizer) -> str:
    import numpy as np, traceback, re
    from nltk.corpus import stopwords

    if shap is None:
        return "SHAP not available. Install `shap` (pip install shap==0.46.0)."

    # Ensure stopwords available
    try:
        STOPWORDS = set(stopwords.words("english"))
    except LookupError:
        import nltk
        nltk.download("stopwords")
        STOPWORDS = set(stopwords.words("english"))

    try:
        # --- Use Hugging Face tokenizer directly
        masker = shap.maskers.Text(tokenizer)

        # --- Scoring function (mean log-prob of generating question given context)
        def score_fn(texts: List[str]) -> np.ndarray:
            scores = []
            for t in texts:
                try:
                    val = _avg_logprob_for_target(model, tokenizer, t[:512], question[:256])
                    scores.append(float(val))
                except Exception:
                    scores.append(-999.0)
            return np.asarray(scores, dtype=float)

        # --- Run SHAP
        explainer = shap.Explainer(score_fn, masker)
        shap_values = explainer([speech[:512]])

        if getattr(shap_values, "values", None) is None:
            return "SHAP returned no values."

        vals = np.abs(shap_values.values[0])
        tokens = shap_values.data[0]
        L = min(len(tokens), len(vals))

        # --- Filter out stopwords & non-alphabetic tokens
        filtered_tokens, filtered_vals = [], []
        for t, v in zip(tokens[:L], vals[:L]):
            clean_t = re.sub(r"[^a-zA-Z]", "", t).lower()
            if clean_t and clean_t not in STOPWORDS and len(clean_t) > 1:
                filtered_tokens.append(t)
                filtered_vals.append(v)

        if not filtered_tokens:
            return "No informative tokens after stopword filtering."

        # --- Normalize values for readability
        filtered_vals = np.array(filtered_vals)
        #filtered_vals /= filtered_vals.max() + 1e-9  # avoid div-by-zero
        filtered_vals /= filtered_vals.sum() + 1e-9


        # --- Sort top tokens
        order = np.argsort(filtered_vals)[::-1][:8]
        top = [f"{filtered_tokens[i]} ({filtered_vals[i]:.3f})" for i in order if filtered_vals[i] > 0]

        if not top:
            return "No positive SHAP signal."

        return "Top influential speech tokens (SHAP, no stopwords): " + ", ".join(top)

    except Exception as e:
        return f"SHAP failed: {type(e).__name__}: {e}\nTraceback:\n{traceback.format_exc()}"




def _attention_explain(text: str, model, tokenizer) -> str:
    """Returns top tokens by last-layer attention weights."""
    import torch
    model.config.output_attentions = True
    enc = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    enc = {k: v.to(next(model.parameters()).device) for k, v in enc.items()}
    with torch.no_grad():
        outputs = model(**enc, output_attentions=True)
    if not hasattr(outputs, "attentions") or outputs.attentions is None:
        return "Attention not available."
    attn = outputs.attentions[-1][0]
    mean_attn = attn.mean(0).mean(0).cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"][0])
    order = np.argsort(mean_attn)[::-1][:8]
    top = [f"{tokens[i]} ({mean_attn[i]:.3f})" for i in order]
    return "Top attention-weighted tokens: " + ", ".join(top)


def explainability_node_all(state: Dict[str, Any], model=None, tokenizer=None) -> Dict[str, Any]:
    """Compute all explainability modes in one go."""
    speech = state.get("speech", "") or ""
    question = state.get("generated_question", "") or ""
    results = {}

    if not speech or not question:
        state["explanation"] = {"error": "Insufficient data for explainability."}
        return state

    try:
        # --- Semantic similarity
        results["semantic"] = _semantic_explain(speech, question)

        # --- SHAP explainability
        try:
            results["shap"] = _shap_explain(speech, question, model, tokenizer)
        except Exception as e:
            results["shap"] = f"SHAP error: {type(e).__name__}: {e}"

        # --- LIME explanation (optional, may be slow)
        if LimeTextExplainer is None:
            results["lime"] = "LIME not available (package missing)."
        else:
            try:
                explainer = LimeTextExplainer()
                exp = explainer.explain_instance(
                    speech,
                    lambda texts: np.array([[np.random.rand(), np.random.rand()] for _ in texts]),
                    num_features=6
                )
                results["lime"] = "LIME placeholder output: " + str(exp.as_list()[:3])
            except Exception as e:
                results["lime"] = f"LIME error: {type(e).__name__}: {e}"

    except Exception as e:
        results["general_error"] = f"Explainability failed: {type(e).__name__}: {e}"

    state["explanation"] = results
    return state


# ============================================================
# 2️⃣ Model loading
# ============================================================
print("🚀 Loading Mistral model...")
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    output_attentions=True
)
print("✅ Model ready on device:", device)


# ============================================================
# 3️⃣ Flask app setup
# ============================================================
app = Flask(__name__)
CORS(app)


# ============================================================
# 4️⃣ Routes
# ============================================================
@app.route("/generate", methods=["POST"])
def generate():
    """Handles text generation for journalist question."""
    data = request.get_json(force=True)
    messages = data.get("messages")
    prompt = data.get("prompt")

    # Support both message-style and legacy prompt input
    if messages is None and prompt:
        messages = [
            {"role": "system", "content": "You are an investigative journalist."},
            {"role": "user", "content": prompt}
        ]

    print("=" * 80)
    print("📩 Generation request received.")
    text = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=600, temperature=0.8, do_sample=True)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("✅ Generated response:", response_text)
    return jsonify({"response": response_text.strip()})





@app.route("/explain", methods=["POST"])
def explain():
    """Handles explainability computation (all modes at once)."""
    data = request.get_json(force=True)
    speech = data.get("speech", "")
    question = data.get("question", "")

    print("=" * 80)
    print("🧩 Explainability request received (all modes)")

    state = {"speech": speech, "generated_question": question}
    explained = explainability_node_all(state, model=model, tokenizer=tokenizer)

    for mode, output in explained["explanation"].items():
        print(f"✅ {mode.upper()} → {output[:200]}{'...' if len(output) > 200 else ''}")

    return jsonify(explained["explanation"])


@app.route("/ping", methods=["GET"])
def ping():
    return jsonify({"status": "alive"})



@app.route("/analyze", methods=["POST"])
def analyze():
    """Analyze the full press conference conversation."""
    data = request.get_json(force=True)
    persona = data.get("persona", "")
    role = data.get("role", "")
    topic = data.get("topic", "")
    speech = data.get("speech", "")
    history = data.get("history", [])

    # --- Build conversation text
    convo = "\n".join([f"{t['role'].capitalize()}: {t['content']}" for t in history])

    prompt = f"""
You are an **AI communication evaluator**.
Analyze the following simulated press conference between a journalist ({persona})
and a guest ({role}) on the topic "{topic}".

Opening speech:
\"\"\"{speech}\"\"\"

Full conversation:
{convo}

---
TASK:
You must produce a *valid JSON object* exactly matching the structure below, with all fields filled in.
Do not include explanations, comments, markdown, or any text outside the JSON.

Each score must be an integer from 0–5, defined as:
0 = incoherent / irrelevant
1 = poor
2 = fair
3 = average
4 = strong
5 = excellent

---
Return **only** this JSON (no text before or after):

{{
  "summary": "2–3 sentences summarizing the conversation.",
  "strengths": ["List 3–5 strong points about the guest's responses."],
  "weaknesses": ["List 3–5 weaknesses or missed opportunities."],
  "suggestions": ["List 3–5 practical recommendations for improvement."],
  "scores": {{
    "clarity": <0–5>,
    "relevance": <0–5>,
    "persuasiveness": <0–5>,
    "consistency": <0–5>,
    "engagement": <0–5>
  }}
}}

<END>
"""



    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    outputs = model.generate(**inputs, max_new_tokens=800, temperature=0.7)
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print("\n=== RAW MODEL OUTPUT ===")
    print(response_text)
    print("=========================\n")
    
    import json, re
    # 1️⃣ Get only the last JSON-like block (after <END>)*
    if "<END>" in response_text:
        response_tail = response_text.split("<END>", 1)[1].strip()
    else:
        response_tail = response_text
        
    # 2️⃣ Extract the last {...} block
    match = re.search(r"\{[\s\S]*\}", response_tail)
    if not match:
        analysis = {"error": "No JSON found", "raw_output": response_tail}
    else:
        json_block = match.group(0)
        
        # 3️⃣ Clean up: remove escaped characters (\n, \") if present
        cleaned = (
            json_block
            .encode('utf-8')
            .decode('unicode_escape')   # converts \n, \t, and \" into real characters
            .replace('\r', '')
            .strip()
        )
        
        # 4️⃣ Parse the cleaned JSON
        try:
            analysis = json.loads(cleaned)
        except Exception as e:
            analysis = {"error": f"Failed to parse cleaned JSON: {e}", "raw_output": cleaned}



    print("\n=== ANALYSIS RESULT ===")
    print(json.dumps(analysis, indent=4, ensure_ascii=False))
    print("========================\n")
    
    return jsonify(analysis)


# ============================================================
# 5️⃣ Start ngrok tunnel
# ============================================================
NGROK_AUTH_TOKEN = "..."  # Replace with your token
!ngrok config add-authtoken {NGROK_AUTH_TOKEN}

public_url = ngrok.connect(5000)
print("✅ Public URL:", public_url.public_url)
print("Use this URL in your local Flask app (api_endpoints.py).")


# ============================================================
# 6️⃣ Run server
# ============================================================
app.run(port=5000)


🚀 Loading Mistral model...


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model ready on device: cuda
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Public URL: https://unbevelled-articularly-linn.ngrok-free.dev
Use this URL in your local Flask app (api_endpoints.py).
 * Serving Flask app '__main__'
 * Debug mode: off


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:48, 48.74s/it]               


✅ SEMANTIC → Likely influential speech parts:
• "We will invest in retraining programs, support ethical AI development, and make sure that every innovation creates opportunities — not unemployment" (sim=0.08)
• "O...
✅ SHAP → Top influential speech tokens (SHAP, no stopwords):  retr (0.143), aining (0.141),  programs (0.081),  way (0.079),  Technology (0.055),  emp (0.042),  humanity (0.036),  work (0.035)
✅ LIME → LIME placeholder output: [('the', 0.04038174553575595), ('support', -0.03513371956093428), ('other', -0.026470773744073257)]


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== RAW MODEL OUTPUT ===

You are an **AI communication evaluator**.
Analyze the following simulated press conference between a journalist (human_interest)
and a guest (Presidential Candidate) on the topic "AI and the Future of Jobs".

Opening speech:
"""My vision for our nation is one where artificial intelligence empowers people, not replaces them. We will invest in retraining programs, support ethical AI development, and make sure that every innovation creates opportunities — not unemployment.

Our administration will launch the National AI Transition Fund, helping companies modernize responsibly while protecting workers. Technology must work for humanity — not the other way around."""

Full conversation:
Journalist: As your administration plans to invest in retraining programs to help workers adjust to the advancements in AI, could you share specific details about the programs you envision, such as who they will be designed for, what skills they will focus on, and who will be resp